# Spread fee

Critical to the calculation of the spread fee are the so-called virtual pools:
- There is a base pool size which is changable using parameters.
- The Terra pool determines the amount of existing Terra stablecoins denominated in SDR. The size of the Terra pool is not stored. Instead, the difference between the base pool and the Terra pool is stored in a variable called terra pool delta.
- The Luna pool is calculated using the constant product:
```
constant_product = base_pool * base_pool
terra_pool = base_pool + terra_pool_delta
luna_pool = constant_product / terra_pool
```

In [1]:
# we start with a base pool size of 50 million sdr and equal pool sized

base_pool = 50_000_000
terra_pool_delta = 0

If someone is to swap TerraSDR to Luna, the amount of Luna to receive is caclulated using the oracle. The spread fee however is calculated by using the constant product:

In [2]:
def calc_luna_pool():
    constant_product = base_pool * base_pool
    terra_pool = base_pool + terra_pool_delta
    return constant_product / terra_pool

def swap_to_luna(amount_terra):
    constant_product = base_pool * base_pool
    terra_pool = base_pool + terra_pool_delta
    luna_pool = constant_product / terra_pool
    
    return luna_pool - constant_product / (terra_pool + amount_terra)

def swap_to_terra(amount_luna):
    constant_product = base_pool * base_pool
    terra_pool = base_pool + terra_pool_delta
    luna_pool = constant_product / terra_pool
    
    return terra_pool - constant_product / (luna_pool + amount_luna)

Given a base pool of 50 million SDR and swapping 1000 TerraSDR gives around 1000 SDR worth of Luna:

In [3]:
swap_to_luna(1000)

999.9800003990531

The difference of around $0.02 has to do with the fact that after swapping, more Luna exists than before while the total amount of Luna has the same value (ie inflation). This effect can be seen more clearly when swapping 10 million TerraSDR:

In [4]:
swap_to_luna(10_000_000)

8333333.333333336

The amount of existing Luna has increased by 20% while the value behind all Luna coins was the same. Thus, 10 million Luna now are worth less than before.

In [5]:
def calc_spread_fee_to_luna(amount_terra):
    amount_luna = swap_to_luna(amount_terra)
    return (amount_terra - amount_luna) / amount_terra

def calc_spread_fee_to_terra(amount_luna):
    amount_terra = swap_to_terra(amount_luna)
    return (amount_luna - amount_terra) / amount_luna

def format(fee):
    fee *= 10000
    fee = int(fee)
    fee /= 100
    return '%.2f%%' % fee

The spread fee for swapping 10 million Terra is around 16%:

In [6]:
format(calc_spread_fee_to_luna(10_000_000))

'16.66%'

In case of swapping Terra to Luna, the terra pool delta is being increased (see [here](https://github.com/terra-rebels/classic-core/blob/main/x/market/keeper/swap.go#L23-L31)). If the terra pool delta is 10 million, this means the terra pool contains 60 million SDR and the luna pool contains around 40 million SDR:

In [7]:
terra_pool_delta = 10_000_000
print('Luna pool %i' % calc_luna_pool())

Luna pool 41666666:


Swapping 10 million Terra results in a significantly higher fee:

In [8]:
format(calc_spread_fee_to_luna(10_000_000))

'40.47%'

At the end of each block, the terra pool delta is lowered. This is done by dividing the terra pool delta by the so-called pool recovery rate:

In [9]:
def adjust_pool_delta(recovery_rate):
    step_size = terra_pool_delta / recovery_rate
    return terra_pool_delta - step_size

When the pool recovery rate is set to 36 and the pool delta is 10 million, at the end of the block the pool delta will have been lowered to around 9.7 million.

In [10]:
terra_pool_delta = adjust_pool_delta(36)
terra_pool_delta

9722222.222222222

At the end of the next block (given no swaps have been made), the pool delta will be around 9.4 million. Thus, if the pool recovery rate is 36, it does not mean that after 36 blocks the delta will be back to 0, but instead the pool delta will slowly converge to 0.

In [11]:
terra_pool_delta = adjust_pool_delta(36)
terra_pool_delta

9452160.49382716

When there has been a swap from Terra to Luna resulting in a big pool delta, the next swap receives a negative spread fee. However, every spread fee lower than the minimum spread fee is set to the minium spread fee (see [here](https://github.com/terra-rebels/classic-core/blob/main/x/market/keeper/swap.go#L127-L129)).

In [12]:
terra_pool_delta = 10_000_000
calc_spread_fee_to_terra(1_000_000)

-0.40625

Swapping from Luna to Terra does not necessarily result in zero fees:

In [13]:
terra_pool_delta = 0
calc_spread_fee_to_terra(10_000_000)

0.1666666666666664

Increasing the base pool size leads to lower spread feeds:

In [14]:
base_pool = 100_000_000
terra_pool_delta = 0
format(calc_spread_fee_to_luna(10_000_000))

'9.09%'